# **저시력자를 위한 원화 화폐 분류**
---
- 본 과제는 UltraLytics YOLO v5 모델 사용을 권장합니다.
    - 본 파일의 목차는 UltraLytics YOLO v5에 맞게 작성되어 있습니다.
    - 다른 모델을 찾아서 사용하셔도 좋습니다.
    - 산출물이 잘 나오면 됩니다 : )
---

## 0.미션
---
- **과제 수행 목표**
    - 본 과제는 Object Detection 문제입니다.
    - Object Detection 문제로 접근하기 위해 **데이터셋 전처리**를 하셔야 합니다.
    - 데이터셋 : money_dataset.zip
        1. 데이터셋은 압축 파일로 제공됩니다.
        2. 압축 파일 안에는 화폐마다 폴더가 개별적으로 존재합니다.
        3. 폴더 안에는 화폐 이미지와 화폐 정보가 담긴 json 파일이 있습니다.
    - 여러분이 직접 촬영한 화폐 사진들을 탐지 과정에서 이용 해보세요.
    - 이미지에 화폐 하나만 나오게 촬영하는 것은 지양해주세요.
    - 다양한 방법으로 화폐를 촬영하고 결과를 확인해보세요.
        - ex 1) 화폐의 모든 종류를 한 이미지에 나오게 촬영
        - ex 2) 여러 화폐를 겹치게 하여 촬영
---
- **Key Point**
    1. 모델에 맞는 폴더 구조 확인
    2. 이미지 축소 비율에 맞춰 좌표값 변경
        - 좌표를 이미지 리사이즈한 비율로 변경
    3. 모델에 맞는 정보 추출/형식 변경
        - json 파일에서 정보 추출 및 모델 형식에 맞게 변경
    4. 화폐당 하나의 클래스로 변경
        - 총 8개 클래스
    5. 모델 선택 필요
---

## 1.환경설정

### (1) 구글 드라이브 연동
---
- 아래의 코드 셀을 반드시 실행시켜야 합니다.
---

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### (2) 데이터셋 불러오기
---
- **세부요구사항**
    - 데이터셋 파일의 압축을 해제하세요.
---
- 예제 코드에서는 zipfile 모듈을 이용하였습니다.
    - [zipfile document](https://docs.python.org/3/library/zipfile.html#zipfile-objects)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [2]:
import zipfile

In [3]:
!pwd

/content


In [4]:
path = '/content/drive/MyDrive/Datasets/'

In [5]:
# 데이터셋 압축 파일 경로 : 유저별로 상이할 수 있음
money_data = zipfile.ZipFile(path + 'money_dataset.zip')

In [6]:
!mkdir /content/Dataset/

In [7]:
# 데이터셋 압축 해제
money_data.extractall('/content/Dataset/money_data/')

## 2.데이터 전처리

### (1) 폴더 구조 생성 및 파일 이동
---
- **세부요구사항**
    -  모델에서 요구하는 폴더 구조를 만들어야 합니다.
        - Hint : Image와 Label을 구분하는 폴더를 만들어 주세요
---
- 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [8]:
# 1.폴더 구조 만들기
!mkdir /content/Dataset/images;
!mkdir /content/Dataset/images/train; mkdir /content/Dataset/images/val

!mkdir /content/Dataset/labels;
!mkdir /content/Dataset/labels/train; mkdir /content/Dataset/labels/val

In [9]:
import glob, shutil

In [10]:
# 2. Dataset metadata 입력
won_list = ['10', '50', '100', '500', '1000', '5000', '10000', '50000']
data_path = '/content/Dataset/'

---
- 데이터를 Training set | Validation set으로 분할하세요.
    - 예시 : Training과 Validation은 8:2로 분리
- Hint : 이미지 데이터는 /images에, JSON 데이터는 /labels에 넣어주세요
    - 예시 : /dataset/images/train, /dataset/labels/train
    - 예제 코드에서는 glob, shutil 모듈을 이용하였습니다.
    - [glob document](https://docs.python.org/3/library/glob.html) | [shutil document](https://docs.python.org/3/library/shutil.html)

    ※ 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
    
---

In [11]:
import os

In [12]:
for won in won_list:
 print(won)

10
50
100
500
1000
5000
10000
50000


In [13]:
# 3. 데이터를 Training set | Validation set으로 분할하세요.
data_path = '/content/Dataset/money_data/'
for won in won_list:
    img_name = os.listdir(data_path + won + '/')
    img_name = [file for file in img_name if file.endswith('.jpg')] # jpg로 끝나는 거
    img_name_val = img_name[:round(len(img_name)*0.2)] # 20%
    img_name_train = img_name[round(len(img_name)*0.2):] # 80%

    img_label = os.listdir(data_path+ won + '/')
    img_label = [file for file in img_label if file.endswith('.json')] # json으로 끝나는 거
    img_label_val = img_label[:round(len(img_label)*0.2)]
    img_label_train = img_label[round(len(img_label)*0.2):]

    for name in img_name_train:
        shutil.copy(data_path + won + '/' + name, '/content/Dataset/images/train/' + name)

    for name in img_name_val:
        shutil.copy(data_path + won + '/' + name, '/content/Dataset/images/val/' + name)

    for name in img_label_train:
        shutil.copy(data_path + won + '/' + name, '/content/Dataset/labels/train/' + name)

    for name in img_label_val:
        shutil.copy(data_path + won + '/' + name, '/content/Dataset/labels/val/' + name)

### (2) json에서 정보 추출
---
- **세부요구사항**
    - json 파일에서 필요한 정보를 추출하세요:
        - 위치 정보 : x1, x2, y1, y2
        - 박스 정보 : shape_type
        - 클래스 정보 : labels
    - 화폐당 하나의 클래스로 변경하세요.
        - json 파일에는 화폐 클래스가 앞뒷면으로 구분되어 있습니다.
        - 화폐의 앞뒷면 구분을 없애주세요.
            - 예시 : 'ten_front', 'ten_back' -> 'ten'
    - 화폐의 위치 정보를 YOLO 모델 형식에 맞게 변경 해주세요.
        - 사용되는 이미지는 원본에서 1/4로 축소되어 있습니다.
        - json 파일의 정보는 원본 기준 데이터이므로 위치 정보 추출을 할 때 x값과 y값을 1/4로 줄여주세요.
    - 이렇게 변경된 정보를 YOLO label 형식에 맞게 txt파일로 저장 해 주세요.
        - Hint : YOLO Labeling Format [label, x-center, y-center, width-norm, height-norm]
---

In [14]:
import os, json

In [15]:
json_path = '/content/Dataset/labels/'
temp_list = ['train', 'val']

In [19]:
!mkdir /content/Dataset/labels/train_txt/
!mkdir /content/Dataset/labels/val_txt/

In [20]:
# YOLO label 형식으로 변환하는 함수
def convert_to_yolo_label(x1, x2, y1, y2, label, image_width, image_height):
    x_center = (x1 + x2) / 2 / 4
    y_center = (y1 + y2) / 2 / 4

    width = ( x2 - x1 ) / 4
    width_norm = width / (image_width / 4)
    height = ( y2 - y1 ) /4
    height_norm = height / (image_height / 4)
    return f"{label} {x_center} {y_center} {width_norm} {height_norm}\n"


# 화폐의 클래스 정보를 변환하는 함수
def convert_currency_label(label):
    if label == 'Ten_front' or label == 'Ten_back': # 10
        return 10
    elif label == 'Fifty_front' or label == 'Fifty_back': # 50
        return 50
    elif label == 'Hundred_front' or label == 'Hundred_back': # 100
        return 100
    elif label == 'Five_Hundred_front' or label == 'Five_Hundred_back': # 500
        return 500
    elif label == 'Thousand_front' or label == 'Thousand_back': # 1000
        return 1000 
    elif label == 'Five_Thousand_front' or label == 'Five_Thousand_back': # 5000
        return 5000  
    elif label == 'Ten_Thousand_front' or label == 'Ten_Thousand_back': # 10000
        return 10000
    elif label == 'Fifty_Thousand_front' or label == 'Fifty_Thousand_back': # 50000
        return 50000
    else:
        return label

# json 파일 불러오기
label_tr_path = json_path + 'train'
label_val_path = json_path + 'val'

label_path = [label_tr_path, label_val_path]
txt_path = [json_path + 'train_txt', json_path + 'val_txt']
for i in range(2):
    for filename in os.listdir(label_path[i]):
        with open(os.path.join(label_path[i], filename), "r") as f:
            data = json.load(f)

        # 필요한 정보 추출
        image_width = data["imageWidth"] 
        image_height = data["imageHeight"] 
        shapes = data["shapes"]

        # YOLO label 형식으로 변환하여 파일에 저장
        ext = os.path.splitext(filename)[0].lower()
        with open(os.path.join(txt_path[i], ext+'.txt'), "w") as f:
            for shape in shapes:
                label = shape["label"]
                points = shape["points"]
                
                # 위치 정보 추출
                x1 = int(points[0][0])
                x2 = int(points[1][0])
                y1 = int(points[0][1])
                y2 = int(points[1][1])

                # 화폐 클래스 정보 변환
                label = convert_currency_label(label)
                # YOLO label 형식으로 변환하여 파일에 저장
                yolo_label = convert_to_yolo_label(x1, x2, y1, y2, label, image_width, image_height)
                f.write(yolo_label)

### (3) 데이터셋 정보가 담긴 파일 생성
---
- **세부요구사항**
    - 파일 안에 있어야 할 정보는 아래와 같습니다.
        - 학습할 클래스 이름 정보
        - 학습할 클래스 수 정보
        - Training, Validation 데이터셋 위치 정보
---
- 가장 대중적으로 이용하는 라이브러리는 yaml 입니다.
    - [yaml document](https://pyyaml.org/wiki/PyYAMLDocumentation)
    - 해당 모듈 이외에 자신이 잘 알고 있는 방법을 사용해도 됩니다.
---

In [21]:
import yaml

In [22]:
won_dict = {0:'10', 1:'50', 2:'100', 3:'500', 4:'1000', 5:'5000', 6:'10000', 7:'50000'}

In [23]:
data = {
    'class_names': [won_dict[i] for i in range(len(won_dict))],
    'num_classes': len(won_dict),
    'datasets': {
        'train': [
            '/content/Dataset/labels/train'
        ],
        'val': [
            '/content/Dataset/labels/val'     
        ]
    }
}

with open('/content/Dataset/money_data.yaml', 'w') as f:
    yaml.dump(data, f)
    

## 3.모델링

### (1) 모델 라이브러리 설치
---

In [ ]:
!pip install jedi

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
## yolov5 폴더 requirements.txt 수정 필요
## setuptools<=64.0.2

temp_str = 'setuptools<=64.0.2\n'

f = open('/content/yolov5/requirements.txt', 'r')
f_str = f.readlines()
f.close()

f2 = open('/content/yolov5/requirements.txt', 'w')

for idx, val in enumerate(f_str) :
    if 'setuptools' in val :
        idx_v = idx
        f_str.remove(val)
        f_str.insert(idx_v, temp_str)

for val in f_str :
    f2.write(val)

f2.close() 

In [ ]:
!cd yolov5; pip install -r requirements.txt

### (2) 가중치 파일 다운로드
---
- **세부요구사항**
    - 모델 개발자가 제공하는 사전 학습 가중치 파일을 다운로드 하세요.
        - 해당 과정이 불필요하다면 넘어가셔도 됩니다!
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################


### (3) 학습 : train.py
---
- **세부요구사항**
    - UltraLytics YOLO v5에는 아래의 데이터가 필요합니다.
        - 데이터셋 정보가 담긴 yaml 파일
        - 사용하려는 모델 구조에 대한 yaml 파일
        - 사용하려는 모델의 가중치 파일
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################


## 4.탐지 : detect.py
---
- **세부요구사항**
    - 학습 과정에서 생성된 가중치 파일을 이용하세요.
    - IoU threshold를 0.25 이하로 설정하세요.
    - confidence threshold를 0.75 이상으로 설정하세요.
---
- 여러분이 **직접 촬영한 화폐 사진과 동영상**을 탐지 과정에 이용하여 결과를 확인하세요.
    - 조건
        1. 화폐의 수를 늘려가며 촬영 해보세요.
            - ex) 50원 하나, 50원 둘, 50원 셋, ...
        2. 화폐의 종류를 늘려가며 촬영 해보세요.
            - ex) 50원 하나와 100원 하나, 50원 하나와 100원 하나와 1000원 하나, ...
        3. 사진은 최소 30장 이상, 동영상은 최소 하나 이상 촬영하여 사용 해보세요.
---

In [ ]:
########################
# 이 셀부터 코드 작성하세요
########################
